In [ ]:
# Install necessary packages in Google Colab
!pip install pathway bokeh --quiet


In [ ]:
import pandas as pd
import pathway as pw
import datetime
import pathway.debug as pw_debug


In [ ]:
# Model 1: Baseline Linear Pricing

class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int

data = pw.demo.replay_csv("parking_stream_model2.csv", schema=ParkingSchema, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
    )
    .with_columns(
        price=10 + 5 * (pw.this.occ_max / pw.this.cap)
    )
)

pw_debug.print_table(delta_window)
pw.run()
